# Mortgage Workflow

This example demonstrates ETL and data conversion operations on mortgage data, and is adapted from [RAPIDS Examples](https://github.com/rapidsai-community/notebooks-contrib/blob/branch-0.14/intermediate_notebooks/E2E/mortgage/mortgage_e2e.ipynb)

To acquire this dataset, please visit [RAPIDS Datasets Homepage](https://docs.rapids.ai/datasets/mortgage-data).
This notebook uses [1 Year dataset](http://rapidsai-data.s3-website.us-east-2.amazonaws.com/notebook-mortgage-data/mortgage_2000.tgz) which is ~3.9GB.

In [1]:
import numpy as np
import pandas as pd
import time
import bodo

## Load Acquistion and Performance Data

In [2]:
@bodo.jit
def load_acquisition_data(year, quarter):
    # read acquisition file
    cols = [
        "loan_id",
        "orig_channel",
        "seller_name",
        "orig_interest_rate",
        "orig_upb",
        "orig_loan_term",
        "orig_date",
        "first_pay_date",
        "orig_ltv",
        "orig_cltv",
        "num_borrowers",
        "dti",
        "borrower_credit_score",
        "first_home_buyer",
        "loan_purpose",
        "property_type",
        "num_units",
        "occupancy_status",
        "property_state",
        "zip",
        "mortgage_insurance_percent",
        "product_type",
        "coborrow_credit_score",
        "mortgage_insurance_type",
        "relocation_mortgage_indicator",
    ]
    dtypes = {
        "loan_id": np.int64,
        "orig_channel": pd.CategoricalDtype(["B", "C", "R"]),
        "seller_name": str,
        "orig_interest_rate": np.float64,
        "orig_upb": np.int64,
        "orig_loan_term": np.int64,
        "orig_ltv": np.float64,
        "orig_cltv": np.float64,
        "num_borrowers": np.float64,
        "dti": np.float64,
        "borrower_credit_score": np.float64,
        "first_home_buyer": pd.CategoricalDtype(["N", "U", "Y"]),
        "loan_purpose": pd.CategoricalDtype(["C", "P", "R", "U"]),
        "property_type": pd.CategoricalDtype(["CO", "CP", "MH", "PU", "SF"]),
        "num_units": np.int64,
        "occupancy_status": pd.CategoricalDtype(["I", "P", "S"]),
        "property_state": pd.CategoricalDtype(
            [
                "AK",
                "AL",
                "AR",
                "AZ",
                "CA",
                "CO",
                "CT",
                "DC",
                "DE",
                "FL",
                "GA",
                "HI",
                "IA",
                "ID",
                "IL",
                "IN",
                "KS",
                "KY",
                "LA",
                "MA",
                "MD",
                "ME",
                "MI",
                "MN",
                "MO",
                "MS",
                "MT",
                "NC",
                "ND",
                "NE",
                "NH",
                "NJ",
                "NM",
                "NV",
                "NY",
                "OH",
                "OK",
                "OR",
                "PA",
                "PR",
                "RI",
                "SC",
                "SD",
                "TN",
                "TX",
                "UT",
                "VA",
                "VI",
                "VT",
                "WA",
                "WI",
                "WV",
                "WY",
            ]
        ),
        "zip": np.int64,
        "mortgage_insurance_percent": np.float64,
        "product_type": pd.CategoricalDtype(["FRM"]),
        "coborrow_credit_score": np.float64,
        "mortgage_insurance_type": np.float64,
        "relocation_mortgage_indicator": pd.CategoricalDtype(["N", "Y"]),
    }
    acq_file = "mortgage_data/acq/Acquisition_" + str(year) + "Q" + str(quarter) + ".txt"
    acq_df = pd.read_csv(
        acq_file, names=cols, delimiter="|", dtype=dtypes, parse_dates=[6, 7]
    )
    return acq_df

In [3]:
@bodo.jit
def load_performance_data(perf_file):
    # read performance file
    cols = [
        "loan_id",
        "monthly_reporting_period",
        "servicer",
        "interest_rate",
        "current_actual_upb",
        "loan_age",
        "remaining_months_to_legal_maturity",
        "adj_remaining_months_to_maturity",
        "maturity_date",
        "msa",
        "current_loan_delinquency_status",
        "mod_flag",
        "zero_balance_code",
        "zero_balance_effective_date",
        "last_paid_installment_date",
        "foreclosed_after",
        "disposition_date",
        "foreclosure_costs",
        "prop_preservation_and_repair_costs",
        "asset_recovery_costs",
        "misc_holding_expenses",
        "holding_taxes",
        "net_sale_proceeds",
        "credit_enhancement_proceeds",
        "repurchase_make_whole_proceeds",
        "other_foreclosure_proceeds",
        "non_interest_bearing_upb",
        "principal_forgiveness_upb",
        "repurchase_make_whole_proceeds_flag",
        "foreclosure_principal_write_off_amount",
        "servicing_activity_indicator",
    ]
    dtypes = {
        "loan_id": np.int64,
        "servicer": str,
        "interest_rate": np.float64,
        "current_actual_upb": np.float64,
        "loan_age": np.float64,
        "remaining_months_to_legal_maturity": np.float64,
        "adj_remaining_months_to_maturity": np.float64,
        "msa": np.float64,
        "current_loan_delinquency_status": np.int32,
        "mod_flag": pd.CategoricalDtype(["N", "Y"]),
        "zero_balance_code": pd.CategoricalDtype(
            ["01", "02", "06", "09", "03", "15", "16"]
        ),
        "foreclosure_costs": np.float64,
        "prop_preservation_and_repair_costs": np.float64,
        "asset_recovery_costs": np.float64,
        "misc_holding_expenses": np.float64,
        "holding_taxes": np.float64,
        "net_sale_proceeds": np.float64,
        "credit_enhancement_proceeds": np.float64,
        "repurchase_make_whole_proceeds": np.float64,
        "other_foreclosure_proceeds": np.float64,
        "non_interest_bearing_upb": np.float64,
        "principal_forgiveness_upb": np.float64,
        "repurchase_make_whole_proceeds_flag": pd.CategoricalDtype(["N", "Y"]),
        "foreclosure_principal_write_off_amount": np.float64,
        "servicing_activity_indicator": pd.CategoricalDtype(["N", "Y"]),
    }
    pdf = pd.read_csv(
        perf_file,
        names=cols,
        delimiter="|",
        dtype=dtypes,
        parse_dates=[1, 8, 13, 14, 15, 16],
    )
    return pdf

## Data Conversion

1. Create features denoting whether a loan_id has ever been delinquent for over 30, 90 and 180 days.
2. Compute features denoting the earliest reported date when a loan_id became delinquent for more than 30, 90 and 180 days.
3. Merge the ever and delinq features table on loan_id
4. Join the performance table with the features table. (delinq and ever features)
5. For every loan_id in a 12 month window compute a feature denoting
    whether it has been delinquent for over 3 months or had an unpaid principal balance.
6. Combines the 12_mon features table with the ever_delinq features tables
7. Combines the grouped table with all features with the original Performance table
    

In [4]:
@bodo.jit
def get_tmp_df(joined_df, y):
    n_months = 12
    tmpdf = joined_df[
        ["loan_id", "timestamp_year", "timestamp_month", "delinquency_12", "upb_12"]
    ]
    tmpdf["josh_months"] = tmpdf["timestamp_year"] * 12 + tmpdf["timestamp_month"]
    tmpdf["josh_mody_n"] = np.floor(
        (tmpdf["josh_months"].astype(np.float64) - 24000 - y) / 12
    )
    tmpdf_d = tmpdf.groupby(["loan_id", "josh_mody_n"], as_index=False)[
        "delinquency_12"
    ].max()
    tmpdf_m = tmpdf.groupby(["loan_id", "josh_mody_n"], as_index=False)["upb_12"].min()
    tmpdf_d["upb_12"] = tmpdf_m["upb_12"]
    tmpdf = tmpdf_d
    tmpdf["delinquency_12"] = (tmpdf["delinquency_12"] > 3).astype(np.int32)
    tmpdf["delinquency_12"] += (tmpdf["upb_12"] == 0).astype(np.int32)
    tmpdf["timestamp_year"] = (
        ((tmpdf["josh_mody_n"] * n_months) + 24000 + (y - 1)) / 12
    ).astype(np.int32)
    tmpdf["timestamp_month"] = np.full_like(tmpdf["timestamp_year"].values, y, np.int8)
    tmpdf.drop(columns=["josh_mody_n"], inplace=True)
    return tmpdf

In [5]:
# This is the value used to fill NaN dates
dt64_fill = np.dtype("datetime64[ns]").type("1970-01-01").astype("datetime64[ns]")

In [6]:
@bodo.jit(cache=True)
def morg_func(year, quarter, perf_file):
    t1 = time.time()
    # read names file
    names_file = "mortgage_data/names.csv"
    names_df = pd.read_csv(
        names_file,
        delimiter="|",
        names=["seller_name", "new"],
        dtype={"seller_name": str, "new": str},
    )

    acq_df = load_acquisition_data(year, quarter)
    pdf = load_performance_data(perf_file)

    print("read time", time.time() - t1)
    t1 = time.time()

    acq_df = acq_df.merge(names_df, how="left", on=["seller_name"])
    acq_df.drop(columns=["seller_name"], inplace=True)
    acq_df["seller_name"] = acq_df["new"]
    acq_df.drop(columns=["new"], inplace=True)

    # create ever features
    everdf = pdf[["loan_id", "current_loan_delinquency_status"]]
    everdf = everdf.groupby("loan_id", as_index=False).max()
    everdf["ever_30"] = (everdf["current_loan_delinquency_status"] >= 1).astype(np.int8)
    everdf["ever_90"] = (everdf["current_loan_delinquency_status"] >= 3).astype(np.int8)
    everdf["ever_180"] = (everdf["current_loan_delinquency_status"] >= 6).astype(
        np.int8
    )
    everdf.drop(columns=["current_loan_delinquency_status"], inplace=True)

    # create delinq features
    delinq_df = pdf[
        ["loan_id", "monthly_reporting_period", "current_loan_delinquency_status"]
    ]
    delinq_30 = (
        delinq_df[delinq_df["current_loan_delinquency_status"] >= 1][
            ["loan_id", "monthly_reporting_period"]
        ]
        .groupby("loan_id", as_index=False)
        .min()
    )
    delinq_30["delinquency_30"] = delinq_30["monthly_reporting_period"]
    delinq_30.drop(columns=["monthly_reporting_period"], inplace=True)
    delinq_90 = (
        delinq_df[delinq_df["current_loan_delinquency_status"] >= 3][
            ["loan_id", "monthly_reporting_period"]
        ]
        .groupby("loan_id", as_index=False)
        .min()
    )
    delinq_90["delinquency_90"] = delinq_90["monthly_reporting_period"]
    delinq_90.drop(columns=["monthly_reporting_period"], inplace=True)
    delinq_180 = (
        delinq_df[delinq_df["current_loan_delinquency_status"] >= 6][
            ["loan_id", "monthly_reporting_period"]
        ]
        .groupby("loan_id", as_index=False)
        .min()
    )
    delinq_180["delinquency_180"] = delinq_180["monthly_reporting_period"]
    delinq_180.drop(columns=["monthly_reporting_period"], inplace=True)
    delinq_merge = delinq_30.merge(delinq_90, how="left", on=["loan_id"])
    delinq_merge["delinquency_90"] = delinq_merge["delinquency_90"].fillna(dt64_fill)
    delinq_merge = delinq_merge.merge(delinq_180, how="left", on=["loan_id"])
    delinq_merge["delinquency_180"] = delinq_merge["delinquency_180"].fillna(dt64_fill)

    # join ever delinq features
    everdf = everdf.merge(delinq_merge, on=["loan_id"], how="left")
    everdf["delinquency_30"] = everdf["delinquency_30"].fillna(dt64_fill)
    everdf["delinquency_90"] = everdf["delinquency_90"].fillna(dt64_fill)
    everdf["delinquency_180"] = everdf["delinquency_180"].fillna(dt64_fill)

    # create joined df
    test = pdf[
        [
            "loan_id",
            "monthly_reporting_period",
            "current_loan_delinquency_status",
            "current_actual_upb",
        ]
    ]
    test["timestamp"] = test["monthly_reporting_period"]
    test.drop(columns=["monthly_reporting_period"], inplace=True)
    test["timestamp_month"] = test["timestamp"].dt.month
    test["timestamp_year"] = test["timestamp"].dt.year
    test["delinquency_12"] = test["current_loan_delinquency_status"]
    test.drop(columns=["current_loan_delinquency_status"], inplace=True)
    test["upb_12"] = test["current_actual_upb"]
    test.drop(columns=["current_actual_upb"], inplace=True)
    test["upb_12"] = test["upb_12"].fillna(999999999)
    test["delinquency_12"] = test["delinquency_12"].fillna(-1)

    joined_df = test.merge(everdf, how="left", on=["loan_id"])
    joined_df["ever_30"] = joined_df["ever_30"].fillna(-1)
    joined_df["ever_90"] = joined_df["ever_90"].fillna(-1)
    joined_df["ever_180"] = joined_df["ever_180"].fillna(-1)
    joined_df["delinquency_30"] = joined_df["delinquency_30"].fillna(dt64_fill)
    joined_df["delinquency_90"] = joined_df["delinquency_90"].fillna(dt64_fill)
    joined_df["delinquency_180"] = joined_df["delinquency_180"].fillna(dt64_fill)

    joined_df["timestamp_year"] = joined_df["timestamp_year"].astype(np.int32)
    joined_df["timestamp_month"] = joined_df["timestamp_month"].astype(np.int32)

    # create_12_mon_features
    tmpdf_1 = get_tmp_df(joined_df, 1)
    tmpdf_2 = get_tmp_df(joined_df, 2)
    tmpdf_3 = get_tmp_df(joined_df, 3)
    tmpdf_4 = get_tmp_df(joined_df, 4)
    tmpdf_5 = get_tmp_df(joined_df, 5)
    tmpdf_6 = get_tmp_df(joined_df, 6)
    tmpdf_7 = get_tmp_df(joined_df, 7)
    tmpdf_8 = get_tmp_df(joined_df, 8)
    tmpdf_9 = get_tmp_df(joined_df, 9)
    tmpdf_10 = get_tmp_df(joined_df, 10)
    tmpdf_11 = get_tmp_df(joined_df, 11)
    tmpdf_12 = get_tmp_df(joined_df, 12)
    testdf = pd.concat(
        [
            tmpdf_1,
            tmpdf_2,
            tmpdf_3,
            tmpdf_4,
            tmpdf_5,
            tmpdf_6,
            tmpdf_7,
            tmpdf_8,
            tmpdf_9,
            tmpdf_10,
            tmpdf_11,
            tmpdf_12,
        ]
    )

    # combine_joined_12_mon
    joined_df.drop(columns=["delinquency_12", "upb_12"], inplace=True)
    joined_df["timestamp_year"] = joined_df["timestamp_year"].astype(np.int32)
    joined_df["timestamp_month"] = joined_df["timestamp_month"].astype(np.int8)
    joined_df = joined_df.merge(
        testdf, how="left", on=["loan_id", "timestamp_year", "timestamp_month"]
    )

    # final_performance_delinquency
    merged = pdf
    merged["timestamp_month"] = merged["monthly_reporting_period"].dt.month
    merged["timestamp_month"] = merged["timestamp_month"].astype(np.int8)
    merged["timestamp_year"] = merged["monthly_reporting_period"].dt.year
    merged["timestamp_year"] = merged["timestamp_year"].astype(np.int32)
    merged = merged.merge(
        joined_df, how="left", on=["loan_id", "timestamp_year", "timestamp_month"]
    )
    perf_df = merged.drop(columns=["timestamp_year", "timestamp_month"])

    final_gdf = perf_df.merge(acq_df, how="left", on=["loan_id"])

    final_gdf["delinquency_12"] = final_gdf["delinquency_12"] > 0
    final_gdf["delinquency_12"] = (
        final_gdf["delinquency_12"].fillna(False).astype(np.int32)
    )
    t2 = time.time()
    print("exec time", t2 - t1)
    return final_gdf

In [7]:
perf_file = "mortgage_data/perf/Performance_2000Q1.txt"
df = morg_func(2000, 1, perf_file)

/home/bodo/Bodo/bodo/submit/worker.py:390: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  res = func(*args, **kwargs)
/home/bodo/Bodo/bodo/submit/worker.py:390: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  res = func(*args, **kwargs)
/home/bodo/Bodo/bodo/submit/worker.py:390: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  res = func(*args, **kwargs)
/home/bodo/Bodo/bodo/submit/worker.py:390: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  

/home/bodo/Bodo/bodo/submit/worker.py:390: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  res = func(*args, **kwargs)
/home/bodo/Bodo/bodo/submit/worker.py:390: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  res = func(*args, **kwargs)
/home/bodo/Bodo/bodo/submit/worker.py:390: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  res = func(*args, **kwargs)
/home/bodo/Bodo/bodo/submit/worker.py:390: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  

/home/bodo/Bodo/bodo/submit/worker.py:390: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  res = func(*args, **kwargs)
/home/bodo/Bodo/bodo/submit/worker.py:390: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  res = func(*args, **kwargs)
/home/bodo/Bodo/bodo/submit/worker.py:390: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  res = func(*args, **kwargs)
/home/bodo/Bodo/bodo/submit/worker.py:390: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  

/home/bodo/Bodo/bodo/submit/worker.py:390: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  res = func(*args, **kwargs)
/home/bodo/Bodo/bodo/submit/worker.py:390: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  res = func(*args, **kwargs)
/home/bodo/Bodo/bodo/submit/worker.py:390: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  res = func(*args, **kwargs)
/home/bodo/Bodo/bodo/submit/worker.py:390: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  

/home/bodo/Bodo/bodo/submit/worker.py:390: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  res = func(*args, **kwargs)
/home/bodo/Bodo/bodo/submit/worker.py:390: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  res = func(*args, **kwargs)
/home/bodo/Bodo/bodo/submit/worker.py:390: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  res = func(*args, **kwargs)
/home/bodo/Bodo/bodo/submit/worker.py:390: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  

/home/bodo/Bodo/bodo/submit/worker.py:390: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  res = func(*args, **kwargs)
/home/bodo/Bodo/bodo/submit/worker.py:390: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  res = func(*args, **kwargs)
/home/bodo/Bodo/bodo/submit/worker.py:390: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  res = func(*args, **kwargs)
/home/bodo/Bodo/bodo/submit/worker.py:390: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  

/home/bodo/Bodo/bodo/submit/worker.py:390: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  res = func(*args, **kwargs)
/home/bodo/Bodo/bodo/submit/worker.py:390: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  res = func(*args, **kwargs)
/home/bodo/Bodo/bodo/submit/worker.py:390: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  res = func(*args, **kwargs)
/home/bodo/Bodo/bodo/submit/worker.py:390: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  

read time 6.380462658000003


exec time 1.8116723039999982


In [8]:
display(df.head())

,loan_id,monthly_reporting_period,servicer,interest_rate,current_actual_upb,loan_age,remaining_months_to_legal_maturity,adj_remaining_months_to_maturity,maturity_date,msa,...,num_units,occupancy_status,property_state,zip,mortgage_insurance_percent,product_type,coborrow_credit_score,mortgage_insurance_type,relocation_mortgage_indicator,seller_name
0,100117450928,2001-01-01,<NA>,7.750,129734.54,13.0,347.0,343.0,1970-01-02 15:03:37.528334096,31100.0,...,1,P,CA,912,NaN,FRM,NaN,NaN,N,<NA>
1,100117450928,2001-05-01,<NA>,7.750,128995.24,17.0,343.0,336.0,1970-01-02 15:03:37.528334672,31100.0,...,1,P,CA,912,NaN,FRM,NaN,NaN,N,<NA>
2,100399254253,2001-06-01,<NA>,8.000,53614.55,17.0,163.0,161.0,1970-01-02 15:03:37.528545648,16860.0,...,1,P,TN,374,NaN,FRM,NaN,NaN,N,<NA>
3,100676177700,2002-01-01,<NA>,8.375,40041.43,23.0,337.0,336.0,1970-01-02 15:03:37.555202560,33100.0,...,1,P,FL,331,30.0,FRM,NaN,1.0,N,<NA>
4,100676177700,2002-05-01,<NA>,8.375,39920.61,27.0,333.0,332.0,1970-01-02 15:03:37.555203136,33100.0,...,1,P,FL,331,30.0,FRM,NaN,1.0,N,<NA>
